## Задачи I уровня
1. Реализовать индексы, повышающие производительность операций вставки и изменения платежей без модификации программных компонент

In [28]:
import datetime
import random
import pyodbc
from faker import Faker

database = 'PaymentData'
user = 'sa'
password = 'reallyStrongPwd123'
cnxn = pyodbc.connect('DSN='+database+';DATABASE='+database+';UID='+user+';PWD='+ password)
cursor = cnxn.cursor()

faker = Faker()

In [29]:
def gener_date(start_time, end_time):
    return datetime.datetime.fromtimestamp(
            random.randint(int(start_time.timestamp()), int(end_time.timestamp()))
        )

In [30]:
#copy of Payment Insertion block from data_generation.ipynb

def payment_insertion(num_of_lines):
    oid_category = []
    query = "SELECT Oid FROM [dbo].[PaymentCategory]"
    participants = cursor.execute(query)
    for participant in participants:
        oid_category.append(participant[0])

    oid_participant = []
    query = "SELECT Oid FROM [dbo].[PaymentParticipant]"
    participants = cursor.execute(query)
    for participant in participants:
        oid_participant.append(participant[0])

    oid_project = []
    query = "SELECT Oid FROM [dbo].[Project]"
    participants = cursor.execute(query)
    for participant in participants:
        oid_project.append(participant[0])

    for _ in range(num_of_lines):
        amount = random.randint(1, 100000)
        category = random.choice(oid_category)
        check_number = random.randint(10000, 99999)
        comment = faker.text(100)
        date = gener_date(datetime.datetime(2010, 1, 1), datetime.datetime.now()).isoformat()
        created_date = datetime.datetime.fromtimestamp(
            int(datetime.datetime.now().timestamp())
        ).isoformat()
        is_authorized = random.randint(0, 1)
        justificaton = faker.text(30)
        number = random.randint(10000, 99999)
        optimistic_lock_field = random.randint(1, 3)
        payee, payer = random.sample(oid_participant, 2)
        project = random.choice(oid_project)

        query = f"""INSERT INTO [dbo].[Payment] ([Oid], [Amount], [Category], [Project], [Justification], [Comment], [Date], [Payer], [Payee],
                    [OptimisticLockField], [GCRecord], [CreateDate], [CheckNumber], [IsAuthorized], [Number]) 
                    VALUES (NEWID(), {amount}, '{category}', '{project}', '{justificaton}', '{comment}', '{date}', '{payer}', '{payee}', 
                    {optimistic_lock_field}, Null, '{created_date}', '{check_number}', {is_authorized}, '{number}')"""
        cursor.execute(query)
    cursor.commit()

На данном этапе считаю целесообразно внимательно проанализировать триггер [T_Payment_AI], который срабатывает при операциях INSERT/UPDATE записей с таблицей [dbo].[Payment].

При срабатывании данного триггера вызываются следующие функции: [dbo].[F_CalculatePaymentParticipantBalance], [dbo].[F_CalculateBalanceByMaterial],
[dbo].[F_CalculateBalanceByWork], [dbo].[F_CalculateProjectBalance].

В данных функциях используются вызовы вида `SELECT <...> FROM <...> WHERE <...>`, где после `WHERE` используются поля различных таблиц. Надо искать те поля, на которых нет индексов.

Функция [dbo].[F_CalculateProjectBalance]: в данной функции полями, которые я ищу, считаются `Payment.GCRecord` (на которое существует индекс) и `PaymentCategory.Name` (на котором индекса нет, необходимо создать).

Функция [dbo].[F_CalculateBalanceByWork]: в данной функции полями, которые я ищу, считаются `Payment.GCRecord` (на которое существует индекс), `Payment.Payer` (на которое существует индекс), `Project.Client` (на которое существует индекс), `AccountType.Name` (на котором индекса нет, необходимо создать), `PaymentCategory.Name` (на котором индекса еще нет, но уже выбрано).

Функция [dbo].[F_CalculateBalanceByMaterial]: в данной функции полями, которые я ищу, считаются `Payment.GCRecord` (на которое существует индекс), `Payment.Payer` (на которое существует индекс), `Project.Client` (на которое существует индекс), `Supplier.ProfitByMaterialAsPayer` (на котором индекса нет, необходимо создать), `AccountType.Name` (на котором индекса еще нет, но уже выбрано), `Supplier.ProfitByMaterialAsPayee` (на котором индекса нет, необходимо создать), `PaymentCategory.ProfitByMaterial` (на котором индекса нет, необходимо создать), `PaymentCategory.Name` (на котором индекса еще нет, но уже выбрано), `PaymentCategory.CostByMaterial` (на котором индекса нет, необходимо создать).

Функция [dbo].[F_CalculatePaymentParticipantBalance]: в данной функции полями, которые я ищу, считаются `Payment.GCRecord` (на которое существует индекс), `PaymentCategory.NotInPaymentParticipantProfit` (на котором индекса нет, необходимо создать), `AccountType.Name` (на котором индекса еще нет, но уже выбрано), `PaymentCategory.Name` (на котором индекса еще нет, но уже выбрано).

В итоге надо создать индексы на следующие поля: `PaymentCategory.Name`,  `AccountType.Name`, `Supplier.ProfitByMaterialAsPayer`, `Supplier.ProfitByMaterialAsPayee`, `PaymentCategory.ProfitByMaterial`, `PaymentCategory.CostByMaterial`, `PaymentCategory.NotInPaymentParticipantProfit`.

Измерим сначала время, затрачиваемое на проведение операций INSERT/UPDATE без добавления индексов:

In [32]:
#need a func for update

def payment_update(num_of_lines):
    oid_project = []
    query = f"""SELECT TOP {num_of_lines} Oid FROM [dbo].[Project]"""
    participants = cursor.execute(query)
    for participant in participants:
        oid_project.append(participant[0])
    
    oid_category = []
    query = "SELECT Oid FROM [dbo].[PaymentCategory]"
    participants = cursor.execute(query)
    for participant in participants:
        oid_category.append(participant[0])

    oid_participant = []
    query = "SELECT Oid FROM [dbo].[PaymentParticipant]"
    participants = cursor.execute(query)
    for participant in participants:
        oid_participant.append(participant[0])

    for project in oid_project:
        amount = random.randint(1, 100000)
        category = random.choice(oid_category)
        check_number = random.randint(10000, 99999)
        comment = faker.text(100)
        justificaton = faker.text(30)
        number = random.randint(10000, 99999)
        payee, payer = random.sample(oid_participant, 2)

        query = f"""UPDATE [dbo].[Payment] SET [Amount] = {amount}, [Category] = '{category}', [Justification] = '{justificaton}',
                [Comment] = '{comment}', [Payer] = '{payer}', [Payee] = '{payee}', [CheckNumber] = {check_number}, [Number] = {number} WHERE [Project] = '{project}'"""
        cursor.execute(query)
    cursor.commit()

In [33]:
beginner_time = datetime.datetime.now()
payment_update(100)
stop_time = datetime.datetime.now()

print('UPDATE 100 строк (без добавления индексов): ', (stop_time - beginner_time).total_seconds())


UPDATE 100 строк (без добавления индексов):  1.175467


In [34]:
query = "DELETE FROM [dbo].[Payment]"
cursor.execute(query)
cursor.commit()

In [35]:
beginner_time = datetime.datetime.now()
payment_insertion(1000)
stop_time = datetime.datetime.now()

print('INSERT 1000 строк (без добавления индексов): ', (stop_time - beginner_time).total_seconds())

INSERT 1000 строк (без добавления индексов):  6.171424


In [36]:
# add indices
indecies = [
    "CREATE NONCLUSTERED INDEX iName_PaymentCategory ON [dbo].[PaymentCategory] (Name)",
    "CREATE NONCLUSTERED INDEX iName_AccountType ON [dbo].[AccountType] (Name)",
    "CREATE NONCLUSTERED INDEX iProfitByMaterialAsPayer_Supplier ON [dbo].[Supplier] (ProfitByMaterialAsPayer)",
    "CREATE NONCLUSTERED INDEX iProfitByMaterialAsPayee_Supplier ON [dbo].[Supplier] (ProfitByMaterialAsPayee)",
    "CREATE NONCLUSTERED INDEX iProfitByMaterial_PaymentCategory ON [dbo].[PaymentCategory] (ProfitByMaterial)",
    "CREATE NONCLUSTERED INDEX iCostByMaterial_PaymentCategory ON [dbo].[PaymentCategory] (CostByMaterial)",
    "CREATE NONCLUSTERED INDEX iNotInPaymentParticipantProfit_PaymentCategory ON [dbo].[PaymentCategory] (NotInPaymentParticipantProfit)"
]

for quiery in indecies:
    cursor.execute(query)
    cursor.commit()

In [37]:
beginner_time = datetime.datetime.now()
payment_update(100)
stop_time = datetime.datetime.now()

print('UPDATE 100 строк (с добавлением индексов): ', (stop_time - beginner_time).total_seconds())

UPDATE 100 строк (с добавлением индексов):  0.763045


In [38]:
query = "DELETE FROM [dbo].[Payment]"
cursor.execute(query)
cursor.commit()

In [39]:
beginner_time = datetime.datetime.now()
payment_insertion(1000)
stop_time = datetime.datetime.now()

print('INSERT 1000 строк (с добавлением индексов): ', (stop_time - beginner_time).total_seconds())

INSERT 1000 строк (с добавлением индексов):  6.176895


In [47]:
#need to try smaller batch for INSERT
indecies = [
    "DROP INDEX iName_PaymentCategory ON [dbo].[PaymentCategory] (Name)",
    "DROP INDEX iName_AccountType ON [dbo].[AccountType] (Name)",
    "DROP INDEX iProfitByMaterialAsPayer_Supplier ON [dbo].[Supplier] (ProfitByMaterialAsPayer)",
    "DROP INDEX iProfitByMaterialAsPayee_Supplier ON [dbo].[Supplier] (ProfitByMaterialAsPayee)",
    "DROP INDEX iProfitByMaterial_PaymentCategory ON [dbo].[PaymentCategory] (ProfitByMaterial)",
    "DROP INDEX iCostByMaterial_PaymentCategory ON [dbo].[PaymentCategory] (CostByMaterial)",
    "DROP INDEX iNotInPaymentParticipantProfit_PaymentCategory ON [dbo].[PaymentCategory] (NotInPaymentParticipantProfit)"
]

for quiery in indecies:
    cursor.execute(query)
    cursor.commit()

query = "DELETE FROM [dbo].[Payment]"
cursor.execute(query)
cursor.commit()

beginner_time = datetime.datetime.now()
payment_insertion(100)
stop_time = datetime.datetime.now()

print('INSERT 100 строк (без добавления индексов): ', (stop_time - beginner_time).total_seconds())

INSERT 100 строк (без добавления индексов):  1.143762


In [48]:
indecies = [
    "CREATE NONCLUSTERED INDEX iName_PaymentCategory ON [dbo].[PaymentCategory] (Name)",
    "CREATE NONCLUSTERED INDEX iName_AccountType ON [dbo].[AccountType] (Name)",
    "CREATE NONCLUSTERED INDEX iProfitByMaterialAsPayer_Supplier ON [dbo].[Supplier] (ProfitByMaterialAsPayer)",
    "CREATE NONCLUSTERED INDEX iProfitByMaterialAsPayee_Supplier ON [dbo].[Supplier] (ProfitByMaterialAsPayee)",
    "CREATE NONCLUSTERED INDEX iProfitByMaterial_PaymentCategory ON [dbo].[PaymentCategory] (ProfitByMaterial)",
    "CREATE NONCLUSTERED INDEX iCostByMaterial_PaymentCategory ON [dbo].[PaymentCategory] (CostByMaterial)",
    "CREATE NONCLUSTERED INDEX iNotInPaymentParticipantProfit_PaymentCategory ON [dbo].[PaymentCategory] (NotInPaymentParticipantProfit)"
]

for quiery in indecies:
    cursor.execute(query)
    cursor.commit()

query = "DELETE FROM [dbo].[Payment]"
cursor.execute(query)
cursor.commit()

beginner_time = datetime.datetime.now()
payment_insertion(100)
stop_time = datetime.datetime.now()

print('INSERT 100 строк (с добавлением индексов): ', (stop_time - beginner_time).total_seconds())

INSERT 100 строк (с добавлением индексов):  0.663321


По итогу для операции UPDATE над 100 строками затраченное время сократилось на 35.0858%. Для операции INSERT сначала было выбрано количество строк 1000 (сколько изначально добавлялось на этапе генерации данных). Как можно наблюдать, для 1000 строк время выполнения операции INSERT с добавлением индексов только увеличилось (на 0.088651%). Тогда я попробовала уменьшить кол-во строк до 100 (как в UPDATE), и время значительно сократилось с добавлением индексов (на 42.0053%). Из чего можно сделать вывод, что добавленные индексы хорошо работают при небоьшом кол-ве строк. При росте кол-ва строк данные индексы не показывают результат.